# Virtual experiments, replicating Lahiri and Bevan 2020

In this notebook, we try and replicate a few of the dopamine experiments in Lahiri and Bevan 2020.

4Hz optogenetical stimulation (5 pulses) yielded approximately 0.3 uM DA concentration, while 20Hz stimulation yielded (5 pulses) on average 0.5 uM DA concentration. Here we model the optogenetic activation by setting the DA concentration to these values. In both cases the duration of the resulting DA pulse was set to 5 seconds. 


## Experiment 1

Current injection driving the dSPN to approximately 10Hz, after 5 seconds 0.3 uM DA is added, then current injection is active for another 5 seconds.

Repeat experiment, but this time current injection drives dSPN to approximately 15Hz, and then after 5 seconds 0.5 uM DA is added.

Compare to Figure 1E, 1G, 1H

In [1]:
import os
from snudda import Snudda

neuron = "dspn"
# neuron = "ispn"

neuron_path = os.path.join("data", neuron)
network_path = os.path.join("networks", "bevan_fig1_bath_current_SBML")

In [2]:
snudda = Snudda(network_path=network_path)
si = snudda.init_tiny(neuron_paths=neuron_path, neuron_names=neuron, number_of_neurons=[1], 
                      random_seed=123)

si.network_data["regions"]["Cube"]["neurons"]["dspn"]["reaction_diffusion"] = "data/JSON/reaction_diffusion_D1.json"
# si.network_data["regions"]["Cube"]["neurons"]["ispn"]["reaction_diffusion"] = "data/JSON/reaction_diffusion_D2.json-updated"


# How the ion channels are modified by DA
si.network_data["regions"]["Cube"]["neurons"][neuron]["modulation"] = "test-modulation.json"

si.network_data["regions"]["Cube"]["neurons"][neuron]["modulation_key"] = "abc"

si.write_json()

snudda.create_network()

Warning, no snudda_data set!
Adding neurons: dspn from dir data/dspn
Writing networks/bevan_fig1_bath_current_SBML/network-config.json
Writing networks/bevan_fig1_bath_current_SBML/network-config.json
Placing neurons
Network path: networks/bevan_fig1_bath_current_SBML
Reading SNUDDA_DATA=None from networks/bevan_fig1_bath_current_SBML/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/data from networks/bevan_fig1_bath_current_SBML/network-synapses.hdf5
No n_putative_points and putative_density, setting n_putative_points = 46
(this must be larger than the number of neurons you want to place)
Generating 46 points for networks/bevan_fig1_bath_current_SBML/mesh/Cube-cube-mesh-2.3159794767993218e-05.obj
Filtering, keeping inside points: 1 / 20
Managed to create 1 putative points within the volume.
  WARNING --> is the volume too small? You can create new cube mesh using create_cube_mesh.py

Example how to use create_cube_mesh.py:
from snudda.place.create_cube_mesh impor

In [3]:
# Free memory
snudda = None

mech_dir = "/home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms"
sample_dt = None # 0.00005

sim_config_on = os.path.join("data", "bevan_fig1_with_DA_sbml.json")
sim_config_off = os.path.join("data", "bevan_fig1_no_DA_sbml.json")

sim_output_neuromodulation_ON = os.path.join(network_path, "simulation", "output_neuromodulation_ON.hdf5")
sim_output_neuromodulation_OFF = os.path.join(network_path, "simulation", "output_neuromodulation_OFF.hdf5")

sim_time = 12
n_workers = 1

In [4]:
run_str_on = f"snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config_on} --mechdir {mech_dir} --enable_rxd_neuromodulation"
print(run_str_on)

snudda simulate networks/bevan_fig1_bath_current_SBML --time 12 --simulation_config data/bevan_fig1_with_DA_sbml.json --mechdir /home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms --enable_rxd_neuromodulation


In [5]:
os.system(run_str_on)

numprocs=1
args.ipython_profile = None
args: Namespace(action='simulate', path='networks/bevan_fig1_bath_current_SBML', network_file=None, input_file=None, output_file=None, time=12.0, snudda_data=None, simulation_config='data/bevan_fig1_with_DA_sbml.json', record_volt=True, randomseed=None, disable_synapses=None, disable_gj=None, mech_dir='/home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms', profile=False, verbose=False, exportCoreNeuron=False, record_all=None, enable_rxd_neuromodulation=True, disable_rxd_neuromodulation=None, ipython_profile=None)
MPI Rank: 0, Size: 1
Using input file None
NEURON mechanisms already compiled, make sure you have the correct version of NEURON modules.
If you delete x86_64, aarch64, arm64 directories (or nrnmech.dll) then you will force a recompilation of the modules.
Reading SNUDDA_DATA=None from networks/bevan_fig1_bath_current_SBML/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/data from networks/bevan_fig1_bath_current_

0

In [6]:
run_str_off = f"snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config_off} --mechdir {mech_dir} "
print(run_str_off)

snudda simulate networks/bevan_fig1_bath_current_SBML --time 12 --simulation_config data/bevan_fig1_no_DA_sbml.json --mechdir /home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms 


In [7]:
os.system(run_str_off)

numprocs=1
args.ipython_profile = None
args: Namespace(action='simulate', path='networks/bevan_fig1_bath_current_SBML', network_file=None, input_file=None, output_file=None, time=12.0, snudda_data=None, simulation_config='data/bevan_fig1_no_DA_sbml.json', record_volt=True, randomseed=None, disable_synapses=None, disable_gj=None, mech_dir='/home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms', profile=False, verbose=False, exportCoreNeuron=False, record_all=None, enable_rxd_neuromodulation=None, disable_rxd_neuromodulation=None, ipython_profile=None)
MPI Rank: 0, Size: 1
Using input file None
NEURON mechanisms already compiled, make sure you have the correct version of NEURON modules.
If you delete x86_64, aarch64, arm64 directories (or nrnmech.dll) then you will force a recompilation of the modules.
Reading SNUDDA_DATA=None from networks/bevan_fig1_bath_current_SBML/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/data from networks/bevan_fig1_bath_current_SB

0

In [12]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG

fig = go.Figure()
for data_type in all_species_data:
    idx = time >= 0.0
    fig.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type, line={"width":4}))

"""
fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800,
                 font={"size":18},  # General font size for all elements
                 legend={"font":{"size":16}},  # Specific font size for legend
                 xaxis={"title":{"font":{"size":20}}, "tickfont":{"size":14}},  # X-axis title and tick labels
                 yaxis={"title":{"font":{"size":20}}, "tickfont":{"size":14}})   # Y-axis title and tick labels
"""
fig.show()

In [13]:
# Reporting plot

import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG

fig = go.Figure()
for data_type in ["D1RDA", "PKAc", "cAMP"]: #all_species_data:
    idx = time >= 0.0
    fig.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type, line={"width":4}))

fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800,
                  font={"size":18},  # General font size for all elements
                  legend={"font":{"size":50}},  # Specific font size for legend
                  xaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}},  # X-axis title and tick labels
                  yaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}})   # Y-axis title and tick labels


fig.show()

fig.write_image("bevan_1-example-da-cascade-1.png", width=1200, height=800)


fig2 = go.Figure()
for data_type in ["cal12_ms.modulation_factor", "cal13_ms.modulation_factor", 
                  "kas_ms.modulation_factor", "kir_ms.modulation_factor", "naf_ms.modulation_factor", ]: #all_species_data:
    idx = time >= 0.0
    data_type_str = data_type.replace("modulation_factor", "modulation")
    fig2.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type_str, line={"width":4}))

fig2.update_layout(xaxis_title="Time (s)", yaxis_title="Modulation factor", width=1000, height=800,
                  font={"size":18},  # General font size for all elements
                  legend={"font":{"size":50}},  # Specific font size for legend
                  xaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}},  # X-axis title and tick labels
                  yaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}})   # Y-axis title and tick labels

fig2.show()
fig2.write_image("bevan_fig1_biochem.png", width=1200, height=800)


In [11]:
from snudda.utils import SnuddaLoadSimulation

nd = SnuddaLoadSimulation(sim_output_neuromodulation_ON)
data_types = nd.list_data_types(0)
all_species_data = nd.get_all_data(neuron_id=0, exclude=["spikes", "voltage"])
time = nd.get_time()
voltage = nd.get_data("voltage", [0])

Loading networks/bevan_fig1_bath_current_SBML/simulation/output_neuromodulation_ON.hdf5
WARNING. Depolarisation block in neuron - neuron_id: (name, parameter_key, morphology_key):
0: (dspn_0, pe6ec2d4b, mf702205f)


In [10]:
# Reporting plot

import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG

fig = go.Figure()
for data_type in ["D2RDA", "PKAc", "cAMP"]: #all_species_data:
    idx = time >= 0.0
    fig.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type, line={"width":4}))

fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800,
                  font={"size":18},  # General font size for all elements
                  legend={"font":{"size":50}},  # Specific font size for legend
                  xaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}},  # X-axis title and tick labels
                  yaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}})   # Y-axis title and tick labels


fig.show()

fig.write_image("example-da-cascade-1.png", width=1200, height=800)


fig2 = go.Figure()
for data_type in ["cal12_ms.modulation_factor", "cal13_ms.modulation_factor", 
                  "kas_ms.modulation_factor", "kir_ms.modulation_factor", "naf_ms.modulation_factor", ]: #all_species_data:
    idx = time >= 0.0
    data_type_str = data_type.replace("modulation_factor", "modulation")
    fig2.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type_str, line={"width":4}))

fig2.update_layout(xaxis_title="Time (s)", yaxis_title="Modulation factor", width=1000, height=800,
                  font={"size":18},  # General font size for all elements
                  legend={"font":{"size":50}},  # Specific font size for legend
                  xaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}},  # X-axis title and tick labels
                  yaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}})   # Y-axis title and tick labels

fig2.show()
fig2.write_image("example-da-cascade-2-effect.png", width=1200, height=800)


KeyError: 'D2RDA'

## Experiment 2

Can we show persistence for 15 minutes? (Compare Figure 2)

## Experiment 3

Current injection for 3 seconds, trigger 2 (4) spikes in dSPN. After DA application (1 second, 0.5u M) spikes come earlier and at a higher frequency.

Compare Figure 4 B, C, D

## Experiment 4

Subthreshold current injection (<10 mV depolarisation). Size of depolarisation not affected by DA.

Suprathreshold current injection, with DA lower threshold, and faster spiking.

Compare Figure 5

In [ ]:
Experiment 5

Compare Figure 6